##### 问题:
我们需要编写代码来处理或遍历一个由许多不同类型的对象组成的复杂数据结构，每
种类型的对象处理的方式都不相同。例如遍历一个树结构，根据遇到的树节点的类型
来执行不同的操作。

##### 解决方案:
本节提到的这个问题常常出现在由大量不同类型的对象组成的数据结构的程序中。为
了说明，假设我们正在编写一个表示数学运算的程序。要实现这个功能，程序中会用
到一些类，示例如下：

In [14]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value


之后，我们可以用这些类来构建嵌套式的数据结构，就像这样：

In [15]:

# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

问题不在创建这些数据结构上，而是在稍后编写处理它们的代码时。例如，给定一个
表达式，程序可能要做很多事情，比如产生输出、生成指令、执行字节码到机器码的
翻译等。

为了能让处理过程变得通用，一种常见的解决方案就是实现所谓的“访问者模式”。我
们需要使用类似下面这样的类：

In [16]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__)) 

要使用这个类，程序员从该类中继承并实现各种 visit_Name()方法，这里的 Name 应该
由节点的类型来替换。例如，如果想对表达式求值，那么可以编写这样的代码：

In [17]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    def visit_Negate(self, node):
        return -node.operand


下面这个例子展示如何使用这个类来计算前面生成的表达式：

In [18]:

e = Evaluator()
e.visit(t4)

0.6

In [19]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)  #存在就获取属性值,不存在返回none
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__)) 
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    def visit_Negate(self, node):
        return -node.operand
    
    

# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)
e = Evaluator()
print(e.visit(t4))

0.6


作为另一个完全不同的例子，下面这个类可以将表达式翻译为堆栈机（stack machine）
上的指令序列：

In [20]:
class StackCode(NodeVisitor):
    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions
    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))
    def binop(self, node, instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction,))
    def visit_Add(self, node):
        self.binop(node, 'ADD')
    def visit_Sub(self, node):
        self.binop(node, 'SUB')
    def visit_Mul(self, node):
        self.binop(node, 'MUL')
    def visit_Div(self, node):
        self.binop(node, 'DIV')
    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))
    def visit_Negate(self, node):
        self.unaryop(node, 'NEG')
    


如何使用这个类呢？示例如下：

In [21]:
s = StackCode()
s.generate_code(t4)

[('PUSH', 1),
 ('PUSH', 2),
 ('PUSH', 3),
 ('PUSH', 4),
 ('SUB',),
 ('MUL',),
 ('PUSH', 5),
 ('DIV',),
 ('ADD',)]

本节涵盖了两个核心思想。首先是设计策略，即把操作复杂数据结构的代码和数据结构本身进行解耦。也就是说，本节中没有任何一个 Node 类的实现有对数据进行操作。
相反，所有对数据的处理都放在特定的 NodeVisitor 类中实现。这种隔离使得代码变得
非常通用。

本节的第二个核心思想在于对访问者类本身的实现。在访问者中，你想根据某些值比
如节点类型来调度不同的处理方法。一种幼稚的做法是会编写大量的 if 语句，就像下
面这样：

In [ ]:
'''
class NodeVisitor:
    def visit(self, node):
        nodetype = type(node).__name__
        if nodetype == 'Number':
            return self.visit_Number(node)
        elif nodetype == 'Add':
            return self.visit_Add(node)
        elif nodetype == 'Sub':
            return self.visit_Sub(node)
'''

但是，很快就会发现这种做法明显行不通。除了非常繁琐之外，运行速度也很慢。如
果想添加或修改要处理的节点类型则会难以维护。相反，如果通过一些小技巧将方法
名构建出来，再利用 getattr()函数来获取方法则会好得多。解决方案中的 generic_visit()不
应该匹配到任何处理方法，它是一种异常回退机制。在本节中，generic_visit()会抛出一
个异常来警告程序员遇到了一个未知的节点类型。

在每个访问者类中，常常会通过对 visit()方法进行递归调用来完成计算。示例如下：


In [ ]:
'''
class Evaluator(NodeVisitor):
    ...
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
'''

正是由于递归才使得访问者类可以遍历整个数据结构。本质上说就是不断调用 visit()
直到到达某个终止节点，比如示例中的 Number。递归和其他操作的确切顺序完全取决
于应用程序。

访问者模式的一个缺点就是需要重度依赖于递归。如果要处理一个深度嵌套的数据结
构，那么有可能会达到 Python 的递归深度限制（查看 sys.getrecursionlimit()的结果）。
要避免这个问题，可以在构建数据结构时做一些特定的选择。例如，可以使用普通的
Python 列表来替代链表，或者在每个节点中聚合更多数据，使得数据变得扁平化而不
是深度嵌套。

In [22]:
import sys
sys.getrecursionlimit()

3000

也可以尝试利用生成器和迭代器实现非递归式的遍历算法

在有关解析和编译的程序中使用访问者模式是非常常见的。在 Python 自带的 ast 模块
中可以找到一个实现。除了可以遍历树结构之外，在遍历的同时还允许对数据结构进
行改写或转换（例如添加节点或移除节点）。具体细节可查看 ast 模块的源码。9.24 节
中展示了一个利用 ast 模块来处理 Python 源代码的例子。